In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Desafio Módulo 2 - ML") \
        .getOrCreate()

spark.version

23/01/31 02:29:32 WARN Utils: Your hostname, Deboras-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
23/01/31 02:29:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/31 02:29:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/31 02:29:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


'3.3.1'

In [2]:
stroke_df = spark.read.csv('./stroke_data.csv', header='True', inferSchema='True')

In [3]:
#Quantos registros existem no arquivo?
stroke_df.count()
# R: 67135

67135

In [4]:
#Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, habilite inferSchema=True. Use a 
# função printSchema() da API de Dataframes.
stroke_df.printSchema()
# R: 12 e 7

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [5]:
stroke_df.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

In [6]:
# No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?
stroke_df.groupBy('stroke').count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



In [7]:
#A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e a seguir use spark.sql para escrever 
# uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type). Quantos pacientes sofreram derrame 
# e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?

stroke_df.createOrReplaceTempView('temp_stroke')
df1 = spark.sql("SELECT work_type, COUNT(stroke) AS total_stroke FROM temp_stroke WHERE stroke = 1 GROUP BY work_type")
df1.show()

+-------------+------------+
|    work_type|total_stroke|
+-------------+------------+
| Never_worked|          85|
|Self-employed|       10807|
|      Private|       23711|
|     children|         520|
|     Govt_job|        5164|
+-------------+------------+



In [8]:
# Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:
stroke_df.createOrReplaceTempView('temp_stroke2')
df_gender = spark.sql("SELECT gender, COUNT(0) AS number FROM temp_stroke2 GROUP By gender")
df_gender.show()

+------+------+
|gender|number|
+------+------+
|Female| 39530|
| Other|    11|
|  Male| 27594|
+------+------+



In [9]:
# Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. 
# Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que:
stroke_df.createOrReplaceTempView('temp_stroke3')
# df_hipert_stroke = spark.sql("SELECT hypertension, stroke, COUNT(stroke) AS total_hyp FROM temp_stroke3 GROUP BY hypertension, stroke")
# df_hipert_stroke.show()

df1 = spark.sql("SELECT hypertension, stroke, COUNT(stroke) AS total, ROUND(COUNT(stroke) * 100 / sum(COUNT(stroke)) OVER (),2) AS percentage FROM temp_stroke3 WHERE hypertension = 1 GROUP BY hypertension, stroke")
df1.show()

# df1 = spark.sql("SELECT hypertension, stroke, COUNT(stroke) AS total FROM temp_stroke3 GROUP BY hypertension, stroke")



df2 = spark.sql("SELECT hypertension, stroke, COUNT(stroke) AS total, ROUND(COUNT(stroke) * 100 / sum(COUNT(stroke)) OVER (),2) AS percentage FROM temp_stroke3 WHERE hypertension = 0 GROUP BY hypertension, stroke")
df2.show()

# A hipertensão, neste conjunto de dados, aumenta a probabilidade de derrame. 


23/01/31 02:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/31 02:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/31 02:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/31 02:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/31 02:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/31 02:29:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/01/31 0

In [10]:
# Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. 
# Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?
stroke_df.createOrReplaceTempView('temp_stroke4')

age_df = spark.sql("SELECT age, COUNT(stroke) FROM temp_stroke4 GROUP BY age ORDER BY count(stroke) DESC")
age_df.show()

+----+-------------+
| age|count(stroke)|
+----+-------------+
|79.0|         3258|
|78.0|         2672|
|80.0|         2141|
|81.0|         2005|
|82.0|         1657|
|63.0|         1294|
|66.0|         1195|
|77.0|         1190|
|74.0|         1184|
|57.0|         1160|
|70.0|         1151|
|76.0|         1088|
|67.0|         1070|
|51.0|         1067|
|65.0|         1046|
|75.0|         1015|
|52.0|         1003|
|58.0|          999|
|59.0|          994|
|61.0|          988|
+----+-------------+
only showing top 20 rows



In [11]:
#Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.
stroke_df.filter((stroke_df.age > 50) & (stroke_df.stroke == 1)).count()

28938

In [12]:
#Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.
stroke_df.createOrReplaceTempView('temp_stroke5')
glic_df = spark.sql("SELECT stroke, AVG(avg_glucose_level) FROM temp_stroke5 GROUP BY stroke")
glic_df.show()

+------+----------------------+
|stroke|avg(avg_glucose_level)|
+------+----------------------+
|     1|    119.95307046938272|
|     0|    103.60273130214506|
+------+----------------------+



In [13]:
#Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?
stroke_df.createOrReplaceTempView('temp_stroke6')
imc_df = spark.sql("SELECT stroke, AVG(bmi) FROM temp_stroke6 GROUP BY stroke")
imc_df.show()

+------+------------------+
|stroke|          avg(bmi)|
+------+------------------+
|     1|29.942490629729495|
|     0|27.989678933253657|
+------+------------------+



### Criando modelo DE Machine Learning

In [14]:
# Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas: idade, 
# BMI, hipertensão, doença do coração, nível médio de glicose. Use o conteúdo da segunda aula interativa para criar e avaliar o modelo.

# Primeiro é necessário tornar o dado legível para ML. Dafaframe, tabela não são a melhor opção. Melhor usar vetores.
# Vector Assembler transforma os dados do DF em vetores legíveis para ML

# Montar o dado para o algorítmo entender

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['age', 'bmi', 'hypertension', 'heart_disease', 'avg_glucose_level'], outputCol='features')


In [15]:
# Criar classificadores ????
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

In [16]:
# Criar pipeline
# Abstração que permite compor as etapas do preocesso de ML
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, classifier])

In [17]:
# Dividir dados em treino e teste

train_data, test_data = stroke_df.randomSplit([0.7, 0.3])

In [18]:
# Função .fit() roda o pipeline intenro do modelo
predictStrokeModel = pipeline.fit(train_data)

In [19]:
# Rodar nos dados de teste

predictions = predictStrokeModel.transform(test_data)
predictions.select('gender', 'age', 'hypertension', 'heart_disease', 'bmi', 'avg_glucose_level', 'rawPrediction', 'prediction', 'stroke').show(5)

+------+----+------------+-------------+----+-----------------+---------------+----------+------+
|gender| age|hypertension|heart_disease| bmi|avg_glucose_level|  rawPrediction|prediction|stroke|
+------+----+------------+-------------+----+-----------------+---------------+----------+------+
|Female|62.0|           0|            0|31.2|            85.52|[2184.0,3394.0]|       1.0|     0|
|Female|82.0|           0|            0|24.0|            234.5|[1181.0,7813.0]|       1.0|     0|
|Female|70.0|           0|            0|24.4|            76.34|[2184.0,3394.0]|       1.0|     1|
|Female|43.0|           1|            0|34.2|            60.12|   [93.0,133.0]|       1.0|     0|
|Female|23.0|           0|            0|23.8|           116.95|[4614.0,4064.0]|       0.0|     1|
+------+----+------------+-------------+----+-----------------+---------------+----------+------+
only showing top 5 rows



In [20]:
#  Verificar acurácia do modelo

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy

# Resposta: Menor que 75% 

0.6943261709080107

In [21]:
stroke_df.columns

['0',
 'gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status',
 'stroke']

In [22]:
# Adicione ao modelo as variáveis categóricas: gênero e status de fumante. Use o conteúdo da aula interativa para lidar 
# com as variáveis categóricas.  A acurácia (qualidade) do modelo aumentou para:

# É necessário categorizar a variável GENDER e smoking_status
#Data type string of column gender is not supported.
#Data type string of column smoking_status is not supported.

from pyspark.ml.feature import StringIndexer, OneHotEncoder

gender_indexer = StringIndexer(inputCol='gender', outputCol='genderIndexer')
gender_encoder = OneHotEncoder(inputCol='genderIndexer', outputCol='genderVector')


smoking_status_indexer = StringIndexer(inputCol='smoking_status', outputCol='smoking_statusIndexer')
smoking_status_encoder = OneHotEncoder(inputCol='smoking_statusIndexer', outputCol='smoking_statusVector')


In [23]:
assembler = VectorAssembler(inputCols=['genderVector', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'smoking_statusVector'], outputCol='features')
classifier = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')
pipeline = Pipeline(stages=[gender_indexer,gender_encoder,smoking_status_indexer, smoking_status_encoder, assembler, classifier])
train_data, test_data = stroke_df.randomSplit([0.7, 0.3])
predictStrokeModel = pipeline.fit(train_data)
predictions = predictStrokeModel.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol='stroke', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
accuracy

0.8371571072319202

In [24]:
#Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu na questão (12)?
decisionTreeModel = predictStrokeModel.stages[5]
decisionTreeModel

list(zip(assembler.getInputCols(), decisionTreeModel.featureImportances.values))
#R: smoking_statusVector

[('genderVector', 0.00023438035326561605),
 ('age', 0.1724908757031725),
 ('hypertension', 0.006007246339692424),
 ('heart_disease', 0.0011331132084078143),
 ('avg_glucose_level', 0.4834705720873366),
 ('bmi', 0.3366638123081251)]

In [25]:
#Qual a profundidade da árvore de decisão da questão (12)?
decisionTreeModel.depth

5

In [26]:
# Ver a árvore:
decisionTreeModel.toDebugString

'DecisionTreeClassificationModel: uid=DecisionTreeClassifier_140037484ae3, depth=5, numNodes=25, numClasses=2, numFeatures=9\n  If (feature 7 in {0.0})\n   If (feature 8 in {0.0})\n    Predict: 0.0\n   Else (feature 8 not in {0.0})\n    If (feature 2 <= 56.5)\n     Predict: 0.0\n    Else (feature 2 > 56.5)\n     If (feature 2 <= 73.5)\n      If (feature 5 <= 73.535)\n       Predict: 0.0\n      Else (feature 5 > 73.535)\n       Predict: 1.0\n     Else (feature 2 > 73.5)\n      Predict: 1.0\n  Else (feature 7 not in {0.0})\n   If (feature 2 <= 66.5)\n    If (feature 2 <= 18.5)\n     If (feature 2 <= 13.5)\n      If (feature 5 <= 199.485)\n       Predict: 1.0\n      Else (feature 5 > 199.485)\n       Predict: 0.0\n     Else (feature 2 > 13.5)\n      Predict: 1.0\n    Else (feature 2 > 18.5)\n     Predict: 1.0\n   Else (feature 2 > 66.5)\n    If (feature 2 <= 73.5)\n     If (feature 6 <= 20.25)\n      If (feature 0 in {0.0})\n       Predict: 0.0\n      Else (feature 0 not in {0.0})\n      

In [27]:
assembler.getInputCols()

['genderVector',
 'age',
 'hypertension',
 'heart_disease',
 'avg_glucose_level',
 'bmi',
 'smoking_statusVector']

In [28]:
#Quantos nodos a árvore de decisão possui?
decisionTreeModel.numNodes

25

In [29]:
decisionTreeModel.numFeatures

9

In [30]:
# Salvar modelo (serializar) - persistência do modelo
basePath = "./"
decisionTreeModel.write().overwrite().save(basePath + "/ML_model_stroke")